Good to add a disclaimer that this project is not sponsored by Airbnb



-Baseline Model - Dummy Classifier
-FSM - Something
(These are due next Tuesday)

MVP
-Presentation (Outline)
-Github
-Notebook



In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 


pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("data/Airbnb_Data.csv")

df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,t,NaN,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,f,100%,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,t,t,100%,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,2015-04-19,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,t,t,100%,2015-03-01,t,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74111 entries, 0 to 74110
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      74111 non-null  int64  
 1   log_price               74111 non-null  float64
 2   property_type           74111 non-null  object 
 3   room_type               74111 non-null  object 
 4   amenities               74111 non-null  object 
 5   accommodates            74111 non-null  int64  
 6   bathrooms               73911 non-null  float64
 7   bed_type                74111 non-null  object 
 8   cancellation_policy     74111 non-null  object 
 9   cleaning_fee            74111 non-null  bool   
 10  city                    74111 non-null  object 
 11  description             74111 non-null  object 
 12  first_review            58247 non-null  object 
 13  host_has_profile_pic    73923 non-null  object 
 14  host_identity_verified  73923 non-null

In [5]:
#At first glance, it seems as if there are no duplicates.

df['id'].value_counts()

11667455    1
1106505     1
11659752    1
20439529    1
4717034     1
           ..
6269649     1
1598886     1
9837993     1
7734698     1
13770756    1
Name: id, Length: 74111, dtype: int64

In [6]:
#Changing columns with dates as string to datetime

df['first_review'] = pd.to_datetime(df['first_review'])
df['host_since'] = pd.to_datetime(df['host_since'])
df['last_review'] = pd.to_datetime(df['last_review'])

In [7]:
print(df['first_review'].min())
print(df['first_review'].max())

2008-11-17 00:00:00
2017-10-05 00:00:00


In [8]:
print(df['host_since'].min())
print(df['host_since'].max())

2008-03-03 00:00:00
2017-10-04 00:00:00


In [9]:
print(df['last_review'].min())
print(df['last_review'].max())

2009-01-21 00:00:00
2017-10-05 00:00:00


A limitation of this dataset is that it seems a bit dated through end of 2017. It seems like the last date in the dataset is 2017-10-05. Using this value, we calculate the length of being a host.

In [10]:
last_date = df['last_review'].max()

In [11]:
#We have created a host experiences (in years) column

df['host_experience'] = [(last_date - date).days/365 for date in df['host_since']]

In [12]:
#We have the log_price but also good to create a price_per_night column

df['price_per_night'] = [np.exp(x) for x in df['log_price']]

In [13]:
#We need to combine the strict categories into one

df['cancellation_policy'].value_counts()

strict             32374
flexible           22545
moderate           19063
super_strict_30      112
super_strict_60       17
Name: cancellation_policy, dtype: int64

In [14]:
#We have reclassified the super strict categories into just strict

df['cancellation_policy'] = df['cancellation_policy'].str.replace('super_strict_30', 'strict')
df['cancellation_policy'] = df['cancellation_policy'].str.replace('super_strict_60', 'strict')

In [15]:
df['cancellation_policy'].value_counts()

strict      32503
flexible    22545
moderate    19063
Name: cancellation_policy, dtype: int64

In [16]:
#We see some entries with null values for bedrooms and bathrooms

df[df['bedrooms'].isna()][['bedrooms', 'bathrooms']].head(20)

,bedrooms,bathrooms
200,NaN,1.0
301,NaN,1.0
2368,NaN,NaN
2842,NaN,NaN
3331,NaN,1.0
6647,NaN,1.0
7155,NaN,1.0
8807,NaN,1.0
10513,NaN,1.0
11584,NaN,1.0


In [17]:
#We will resort to dropping the null values in the bedrooms and bathrooms columns.
#We start with bathrooms because the column has more null values.

df.dropna(subset=['bathrooms'], inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73911 entries, 0 to 74110
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      73911 non-null  int64         
 1   log_price               73911 non-null  float64       
 2   property_type           73911 non-null  object        
 3   room_type               73911 non-null  object        
 4   amenities               73911 non-null  object        
 5   accommodates            73911 non-null  int64         
 6   bathrooms               73911 non-null  float64       
 7   bed_type                73911 non-null  object        
 8   cancellation_policy     73911 non-null  object        
 9   cleaning_fee            73911 non-null  bool          
 10  city                    73911 non-null  object        
 11  description             73911 non-null  object        
 12  first_review            58087 non-null  dateti

In [19]:
#Removing null values in the bathrooms category helped but there are still about 86 null values in the bedrooms column.
#We will also remove the null values in bedrooms

df.dropna(subset=['bedrooms'], inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73825 entries, 0 to 74110
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      73825 non-null  int64         
 1   log_price               73825 non-null  float64       
 2   property_type           73825 non-null  object        
 3   room_type               73825 non-null  object        
 4   amenities               73825 non-null  object        
 5   accommodates            73825 non-null  int64         
 6   bathrooms               73825 non-null  float64       
 7   bed_type                73825 non-null  object        
 8   cancellation_policy     73825 non-null  object        
 9   cleaning_fee            73825 non-null  bool          
 10  city                    73825 non-null  object        
 11  description             73825 non-null  object        
 12  first_review            58019 non-null  dateti

In [21]:
df['city'].value_counts()

NYC        32205
LA         22360
SF          6413
DC          5672
Chicago     3716
Boston      3459
Name: city, dtype: int64

In [22]:
#We will be working with the Airbnb listings of that are only in NYC

df_nyc = df[df['city'] == 'NYC'].reset_index(drop=True)

In [23]:
df_nyc.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,host_experience,price_per_night
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,t,NaN,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0,5.531507,150.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,f,100%,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0,0.295890,169.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,t,t,100%,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0,0.945205,145.0
3,5578513,4.605170,Apartment,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,1.0,Real Bed,strict,True,NYC,This is a bright bedroom in an historic buildi...,2013-04-28,t,t,100%,2013-03-27,f,2016-11-15,40.723883,-73.983880,Large East Village Bedroom To Let!,Alphabet City,82,93.0,https://a0.muscache.com/im/pictures/21726900/1...,10009.0,1.0,1.0,4.528767,100.0
4,17589436,4.882802,Apartment,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",2,1.0,Real Bed,strict,True,NYC,"I am renting a very spacious, unique, bright L...",2015-09-26,t,t,71%,2015-08-17,f,2016-12-12,40.719086,-73.990285,"Amazing LES apt - cool, bright...",Lower East Side,26,86.0,NaN,10002,1.0,2.0,2.136986,132.0


In [24]:
df_nyc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32205 entries, 0 to 32204
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      32205 non-null  int64         
 1   log_price               32205 non-null  float64       
 2   property_type           32205 non-null  object        
 3   room_type               32205 non-null  object        
 4   amenities               32205 non-null  object        
 5   accommodates            32205 non-null  int64         
 6   bathrooms               32205 non-null  float64       
 7   bed_type                32205 non-null  object        
 8   cancellation_policy     32205 non-null  object        
 9   cleaning_fee            32205 non-null  bool          
 10  city                    32205 non-null  object        
 11  description             32205 non-null  object        
 12  first_review            25367 non-null  dateti

In [25]:
df_nyc['review_scores_rating'].value_counts()

100.0    7093
93.0     1721
96.0     1672
95.0     1555
97.0     1529
98.0     1485
90.0     1451
94.0     1194
80.0     1151
92.0      988
99.0      752
91.0      745
89.0      548
87.0      533
88.0      507
85.0      292
86.0      275
60.0      246
84.0      212
83.0      189
82.0      100
70.0       91
73.0       75
81.0       61
40.0       53
75.0       50
20.0       42
78.0       42
79.0       42
77.0       39
76.0       38
67.0       29
74.0       22
72.0       13
65.0       12
50.0       12
68.0       10
69.0        7
71.0        6
64.0        5
53.0        3
57.0        3
63.0        2
47.0        2
66.0        2
62.0        2
55.0        2
56.0        1
27.0        1
Name: review_scores_rating, dtype: int64

In [ ]:
#observing null score ratings

df_nyc_nullscores = df_nyc[df_nyc['review_scores_rating'].isnull() == True][['host_since','number_of_reviews', 'last_review', 'review_scores_rating']]

In [ ]:
#The vast majority of null scores have 0 or 1 reviews

df_nyc_nullscores['number_of_reviews'].value_counts()

In [ ]:
#It's important to understand though that there are listing with no review scores that had more than one review.

df_nyc_nullscores[df_nyc_nullscores['number_of_reviews'] > 1]

It is difficult to tell why there are no review scores associated with listings with more than 1 review. However, these listings could be associated with suspended listings. 

"Airbnb has a strict policy on banning hosts who violate their community standards or terms of service. The company reserves the right to remove a host's listing and suspend or terminate their account if they engage in behavior that is deemed inappropriate, including but not limited to discrimination, harassment, fraudulent activity, or unsafe or illegal conduct."

Given the lack of information about the reviews/ratings for newer listings and potential suspended listings, we remove them from the dataset??????????


Or do we keep them in the dataset and assign a value ?????????

We'll come back to this

For those with no review - we will assign them the mean score
For those with more than 1 review but no review score - we will assume that the host lost credibility


In [ ]:
### This code is not working properly. Need to fix

mean_rating = df_nyc['review_scores_rating'].mean()

# Fill null values based on conditions
df_nyc['review_scores_rating'] = df_nyc.apply(
    lambda row: mean_rating if row['number_of_reviews'] in [0, 1] else 0
    if pd.isnull(row['review_scores_rating']) else row['review_scores_rating'],
    axis=1
)

In [ ]:
df_nyc['review_scores_rating'].value_counts()

In [ ]:
#The amenities column needs a lot of cleaning up.

df_nyc['amenities'][0]

In [ ]:
#Cleaning up column to remove {} and ""

df_nyc['amenities'] = df_nyc['amenities'].str.replace('{', '').str.replace('}', '').str.replace('"', '')

In [ ]:
df_nyc['amenities'][0]

In [ ]:
#First we will look at how many unique amenities exist

# Step 1: Extract the 'amenities' column
amenities_column = df_nyc['amenities']

# Step 2: Create an empty set
unique_amenities = set()

# Step 3: Iterate over each row
for row in amenities_column:
    # Step 4: Split the string value by commas
    amenities_list = row.split(',')

    # Step 5: Iterate over each amenity
    for amenity in amenities_list:
        # Step 6: Remove leading and trailing spaces
        cleaned_amenity = amenity.strip()

        # Step 7: Add the cleaned amenity to the set
        unique_amenities.add(cleaned_amenity)

# Step 8: Print the unique amenities
print(unique_amenities)

In [ ]:
#Have identified 118 unique amenities but the output needs some cleaning

len(unique_amenities)

In [ ]:
#We are cleaning up the Unique Amenities
unique_amenities.remove('')
unique_amenities.remove('translation missing: en.hosting_amenity_49')
unique_amenities.remove('translation missing: en.hosting_amenity_50')

In [ ]:
len(unique_amenities)

In [ ]:
unique_amenities_list = sorted(unique_amenities)

In [ ]:
unique_amenities_list

In [ ]:
#Now we create an amenities_count column to figure out how many amenities exist for each listing
df_nyc['amenities_count'] = 0
row_count = 0

for row in df_nyc['amenities']:
    amenities_list = row.split(',')
    cleaned_list = []
    count = 0

    for amenity in amenities_list:
        cleaned_amenity = amenity.strip()
        cleaned_list.append(cleaned_amenity)

    for amenity in cleaned_list:
        if amenity in unique_amenities_list:
            count += 1

    df_nyc.loc[row_count, 'amenities_count'] = count
    row_count += 1

In [ ]:
#Checking if the amenities_count was done correctly

df_nyc[['amenities','amenities_count']].head()

In [ ]:
#We have eight unique amenities excluding the last value that doesn't tell us anything. The count is working correctly.

df_nyc['amenities'][0]

In [ ]:
#

df_nyc['amenities_count'].value_counts().head(20)

-Explored but might not be necessary....

We need to narrow down the unique amenities to figure out which ones are amenities that guests want

-https://www.airbnb.com/resources/hosting-homes/a/the-amenities-guests-want-25
-https://www.superhog.com/blog/must-have-airbnb-amenities/

Airbnb commissioned a survey of 8,629 individuals across the US, Canada, Australia, China, Mexico, Brazil, India, Germany, Spain, Argentina and Italy ages 18 and over via Pollfish in July 2018 to better understand what amenities modern travelers crave most and how they differ by location and age
-https://news.airbnb.com/amenities-do-matter-airbnb-reveals-which-amenities-guests-search-for-most/

Total Amenities (Count)
Desired Amenities

In [ ]:
df_nyc.info()

We will now work on developing a host reliability score. We will use the following columns to construct such column:
- host_has_profile_pic
- host_identity_verified
- host_response_rate
- host_experience

The host reliability score will be created using an equal weighting of each column to come up with a score.
- 25% x host_has_profile_pic + 25% x host_identity_verified + 25% x host_response_rate + 25% x host_experience_percentile

To start we will start by cleaning up the host_has_profile_pic and host_identify_verified columns, which has 't' and 'f' as values. We will convert these into 1 and 0, respectively.

In [ ]:
df_nyc['host_has_profile_pic'] = df_nyc['host_has_profile_pic'].replace({'t': 1, 'f': 0})
df_nyc['host_identity_verified'] = df_nyc['host_identity_verified'].replace({'t': 1, 'f': 0})

In [ ]:
df_nyc.info()

In [ ]:
#We move on to cleaning up the host_response_rate, which consists of a good amount of null values

df_nyc[df_nyc['host_response_rate'].isna()][['host_response_rate','number_of_reviews']].head(5)

#What we can see is that host_response_rate is not necessarily because the hosts are new.
#It is more likely that the null values in the host_response_rate are associated with no responses.

In [ ]:
df_nyc['host_response_rate'].value_counts()

In [ ]:
#For the null values, we assign 0% to hosts with null value response rates.
#However, this column is currently an object. As such, we change the column into a type float

df_nyc['host_response_rate'].fillna('0%', inplace=True)
df_nyc['host_response_rate'] = df_nyc['host_response_rate'].str.rstrip('%').astype(float) / 100

In [ ]:
df_nyc.head()

In [ ]:
#Finally, we take a look at host experience. We first check for any null values

df_nyc[df_nyc['host_experience'].isna()][['id','number_of_reviews','host_experience']]

In [ ]:
#We see that these values are not available and it is not because the hosts are new.
#Given that it is only 176 rows, we will resort to dropping

df_nyc.dropna(subset=['host_experience'], inplace=True)

In [ ]:
df_nyc.info()

In [ ]:
#We now create a new column host_experience_percentile, which is the percentile ranking of the host experience
df_nyc['host_experience_percentile'] = stats.rankdata(df_nyc['host_experience'], method='average') / len(df_nyc)


In [ ]:
df_nyc.head(20)

In [ ]:
#We are now ready to create our host_reliability_score column
#25% x host_has_profile_pic + 25% x host_identity_verified + 25% x host_response_rate + 25% x host_experience_percentile

weights = [0.25, 0.25, 0.25, 0.25]

df_nyc['host_reliability_score'] = np.average(df_nyc[['host_has_profile_pic', 
                                                      'host_identity_verified', 
                                                      'host_response_rate',
                                                      'host_experience_percentile']],
                                              axis=1,
                                              weights=weights
                                             )

In [ ]:
df_nyc.head()

In [ ]:
df_nyc.info()

In [ ]:
#We now create a new column log_price_percentile, which is the percentile ranking of log_price column
#df_nyc['log_price_percentile'] = stats.rankdata(df_nyc['log_price'], method='average') / len(df_nyc)

Feature Engineering Ideas
- host_has_profile_pic	host_identity_verified	host_response_rate
Use the columns to create a host reliability metric

-look at histogram of price or log_price, amenities

-feature engineering something with beds & baths?

pricing percentile - standardized column

-good deal, neutral deal, great deal target value
-map out the good deals, bad deals, and neutral deals across the NYC boroughs?